In [2]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import io
import json
import pandas as pd
from pandasql import sqldf
import zipfile
import regex
from collections import Counter


ID ="1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"
CREDS_PATH = "../develop/creds.json"
FILENAME = "tweets"


with open(CREDS_PATH, 'r') as file:
    credz = json.load(file)


def download_file(credz, file_name):
    credentials = service_account.Credentials.from_service_account_info(credz)
    drive_service = build('drive', 'v3', credentials=credentials)
    request = drive_service.files().get_media(fileId=ID)
    fh = io.FileIO(f'{file_name}.zip', 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()


def convert_to_parquet(file_name):
    # Replace 'file_path.zip' with the path to your zipped file
    with zipfile.ZipFile(f'{file_name}.zip', 'r') as z:
        # Assuming there's only one file in the zip archive
        file_in_zip = z.namelist()[0]
        with z.open(file_in_zip) as f:
            # Load the JSON file into a DataFrame
            df = pd.read_json(f, lines=True)
    df.to_parquet(f'{file_name}.parquet')


download_file(credz, FILENAME)
convert_to_parquet(FILENAME)
df = pd.read_parquet(f'../develop/{FILENAME}.parquet')

In [15]:
def q1(df: pd.DataFrame) -> list:
    # Converting 'date' to just date (without time) and extracting 'username' from the 'user' dictionary
    df['date'] = pd.to_datetime(df['date']).dt.date
    # Extract 'username' from the dictionary in the 'user' column (assuming 'user' is a dictionary)
    df['username'] = df['user'].apply(lambda x: x.get('username', 'unknown') if isinstance(x, dict) else 'unknown')
    columns = ["date", "username"]
    df_selected = df[columns]
    # Proceed with the SQL query, using only 'date' and 'username' columns
    query = """
    SELECT date, username, COUNT(*) as post_count
    FROM df_selected
    GROUP BY date, username
    ORDER BY date DESC, post_count DESC
    """
    # Executing the query using pandasql
    result = sqldf(query)
    result['date'] = pd.to_datetime(result['date']).dt.date
    result_list = [(result['date'][i], result['username'][i]) for i in range(10)]
    return result_list


def count_emojis(text):
    # Regex to get emojis
    emoji_pattern = regex.compile("["
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U0001F300-\U0001F5FF"  # Symbols & Pictograms
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F680-\U0001F6FF"  # Transport & Maps
        "\U0001F700-\U0001F77F"  # Alchemical Symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictograms
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Extended-A Symbols and Pictograms
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed Characters
        "]+", flags=regex.UNICODE)
    # Find emojis
    emojis = emoji_pattern.findall(text)
    individual_emojis = []
    # Iterate over the found emojis
    for emoji in emojis:
        # Check if the emoji is a sequence of the same emoji
        if len(emoji) == 1:
            individual_emojis.append(emoji)
        else:
            # Split the sequence into individual emojis and add them to the list
            individual_emojis.extend(list(emoji))
    return individual_emojis

def q2(df, column='content'):
    # Pass regex to find emojis
    emoji_list = []
    for text in df['content'].dropna():
        emoji_list.extend(count_emojis(text))
    # Count and get most common emojis
    top_emojis_manual = Counter(emoji_list).most_common(10)
    return top_emojis_manual

def q3(df, column='content'):
    # Define the regular expression to identify user mentions
    user_mention_regexp = regex.compile(r'@(\w+)', regex.UNICODE)
    # Initialize a counter for user mentions
    mentions_counter = Counter()
    # Iterate through each text in the DataFrame to count the mentions
    for text in df[column].dropna():
        mentioned_users = user_mention_regexp.findall(text)
        mentions_counter.update(mentioned_users)
    # Obtain the top 10 most mentioned users
    top_mentioned_users_list = mentions_counter.most_common(10)
    return top_mentioned_users_list

In [16]:
q1(df)

[(datetime.date(2021, 2, 24), 'preetysaini321'),
 (datetime.date(2021, 2, 24), 'AhluwaliaA2'),
 (datetime.date(2021, 2, 24), 'JBBal75'),
 (datetime.date(2021, 2, 24), 'rumsomal'),
 (datetime.date(2021, 2, 24), 'ajityadavdu'),
 (datetime.date(2021, 2, 24), 'MaanDee08215437'),
 (datetime.date(2021, 2, 24), 'shells_n_petals'),
 (datetime.date(2021, 2, 24), 'MalikAvni'),
 (datetime.date(2021, 2, 24), 'Sangha_jass11'),
 (datetime.date(2021, 2, 24), 'jas04361527')]